In [ ]:
!pip install -qqq datasets

In [ ]:
from datasets import load_dataset

# Load the default config
dataset = load_dataset("code-search-net/code_search_net")

# List available splits
print(dataset.keys())


In [ ]:
dataset

In [ ]:
dataset["test"]

In [ ]:
dataset["test"]["whole_func_string"][0]

In [ ]:
dataset["test"]["func_code_string"][0]

In [ ]:
# Convert test split to pandas for analysis
df = dataset["test"].to_pandas()

# Show basic info
print(df.info())

# Show a sample row
print(df.sample(1).T)


In [ ]:
!pip install -qqq matplotlib

In [ ]:
import matplotlib.pyplot as plt

# Count per language
lang_counts = df['language'].value_counts()
langs = lang_counts.index.tolist()
counts = lang_counts.values.tolist()

# Bar chart with counts
plt.figure(figsize=(10, 6))
bars = plt.bar(langs, counts)
plt.title("Function Count per Language (Test Set)")
plt.xlabel("Language")
plt.ylabel("Number of Functions")

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 500, f'{height}', ha='center', va='bottom')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Pie chart with counts included in labels
plt.figure(figsize=(8, 8))
labels = [f'{l} ({c})' for l, c in zip(langs, counts)]
plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=140)
plt.title("Language Distribution (Test Set)")
plt.axis('equal')
plt.tight_layout()
plt.show()


In [ ]:
# Compute stats using whole_func_string (not func_code_string)
df['line_count'] = df['whole_func_string'].apply(lambda x: len(x.split('\n')))
df['char_count'] = df['whole_func_string'].apply(len)
df['token_count'] = df['func_code_tokens'].apply(len)

# Print summaries
print("Function Length (Lines - whole function):\n", df['line_count'].describe())
print("\nFunction Length (Characters - whole function):\n", df['char_count'].describe())
print("\nFunction Length (Tokens - from body only):\n", df['token_count'].describe())

# Plot histograms
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.hist(df['line_count'], bins=50, color='skyblue')
plt.title("Lines per Function (whole_func_string)")
plt.xlabel("Lines")
plt.ylabel("Count")

plt.subplot(1, 3, 2)
plt.hist(df['char_count'], bins=50, color='salmon')
plt.title("Characters per Function (whole_func_string)")
plt.xlabel("Characters")

plt.subplot(1, 3, 3)
plt.hist(df['token_count'], bins=50, color='seagreen')
plt.title("Tokens per Function (func_code_tokens)")
plt.xlabel("Tokens")

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import random

# Detect all unique languages in the test set
languages = sorted(df["language"].dropna().unique().tolist())
print(f"Languages found in test set: {languages}")

In [ ]:
samples_per_bucket = 17  # 3 buckets → 51 total per language
final_samples = []

for lang in languages:
    print(f"\nProcessing language: {lang}")

    # Filter data for current language
    df_lang = df[df["language"] == lang].copy()

    # Filter out rows with empty or whitespace-only functions
    df_lang = df_lang[df_lang["whole_func_string"].str.strip().astype(bool)]

    # Add word length column (can also use line/char count instead)
    df_lang["word_len"] = df_lang["whole_func_string"].apply(lambda x: len(x.split()))

    if len(df_lang) < 100:
        print(f"Skipped {lang}: not enough valid samples (only {len(df_lang)} after filtering).")
        continue

    # Sort and split into buckets
    sorted_df = df_lang.sort_values("word_len").reset_index()
    n = len(sorted_df)

    short_df = sorted_df.iloc[:n // 3]
    medium_df = sorted_df.iloc[n // 3: 2 * n // 3]
    long_df = sorted_df.iloc[2 * n // 3:]

    try:
        short_sample = short_df.sample(n=samples_per_bucket, random_state=42).copy()
        medium_sample = medium_df.sample(n=samples_per_bucket, random_state=42).copy()
        long_sample = long_df.sample(n=51 - 2 * samples_per_bucket, random_state=42).copy()
    except ValueError as e:
        print(f"Skipped {lang}: sampling error - {e}")
        continue

    short_sample["length_bucket"] = "short"
    medium_sample["length_bucket"] = "medium"
    long_sample["length_bucket"] = "long"

    language_sample = pd.concat([short_sample, medium_sample, long_sample])
    final_samples.append(language_sample)

# Combine all samples
if final_samples:
    df_sampled = pd.concat(final_samples).reset_index()
    print(f"\nTotal samples collected: {len(df_sampled)}")
    print("Samples per language:")
    print(df_sampled["language"].value_counts())
else:
    print("No samples were collected. Check filtering or data availability.")

# Save to file (JSON Lines format for downstream processing)
df_sampled.to_json("stratified_code_samples_test.jsonl", orient="records", lines=True)


In [ ]:
df_sampled.to_json("sampled_code_snippets.json", orient="records", indent=2)


In [ ]:
!pip install -U transformers accelerate


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Step 1: Load model and tokenizer (Gemma or any other)
def load_model(model_id="google/gemma-2-9b-it"):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    dtype = torch.bfloat16 if device == "cuda" else torch.float32

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=dtype
    ).to(device)

    return model, tokenizer, device


In [ ]:
def generate_summary(code, model, tokenizer, device, language_name="English", max_new_tokens=500, temperature=0.1):

    user_prompt = (
        f"Please generate a code summary for the given code snippet in {language_name}. "
        "Include the following information: the function's name, a brief description of its purpose, "
        "a list of its arguments and their types, and a summary of its key logic — all in plain, natural text without formatting or section headings.\n\n"
        f"\n\nThe code is:\n\n{code}\n\n"
        )


    messages = [{"role": "user", "content": user_prompt}]
    model_input = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer.encode(model_input, add_special_tokens=False, return_tensors="pt").to(device)

    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        top_p=0.1
    )
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=False)

    if "<start_of_turn>model" in output_text:
        clean_summary = output_text.split("<start_of_turn>model")[-1].split("<end_of_turn>")[0].strip()
    else:
        clean_summary = output_text.strip()

    return clean_summary


In [ ]:
target_natural_languages = ["English", "Chinese", "French", "Spanish", "Portuguese", "Arabic", "Hindi"]


In [ ]:
import json
from tqdm import tqdm

def summarize_and_save_per_language(df_sampled, model, tokenizer, device, model_name="gemma-2-9b-it"):
    all_results = []

    for prog_lang in df_sampled['language'].unique():
        df_lang = df_sampled[df_sampled['language'] == prog_lang]
        results = []

        print(f"\nGenerating summaries for: {prog_lang}")

        for row in tqdm(df_lang.itertuples(index=False), total=len(df_lang), leave=False):
            entry = {
                "id": f"sample_{row.index}",
                "language": prog_lang,
                "length_bucket": getattr(row, "length_bucket", "unknown"),  # <-- Added this line
                "code": row.whole_func_string,
                "docstring": getattr(row, "func_documentation_string", ""),
                "model_name": model_name
            }

            for lang in target_natural_languages:
                summary = generate_summary(row.whole_func_string, model, tokenizer, device, language_name=lang)
                entry[f"summary_{lang.lower()}"] = summary

            results.append(entry)
            all_results.append(entry)

        filename = f"{prog_lang.lower()}_summary_all_languages_{model_name.replace('/', '-')}.json"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(results, f, ensure_ascii=False, indent=2)

        print(f"Saved per-language file: {filename}")

    # Save the combined big file
    big_filename = f"all_languages_combined_{model_name.replace('/', '-')}.json"
    with open(big_filename, "w", encoding="utf-8") as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2)

    print(f"\nSaved big combined file: {big_filename}")


In [ ]:
model, tokenizer, device = load_model("google/gemma-2-9b-it")

summarize_and_save_per_language(
    df_sampled=df_sampled,
    model=model,
    tokenizer=tokenizer,
    device=device,
    model_name="gemma-2-9b-it"
)